In [15]:
# IMPORT & SETUP
import os
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime

In [16]:
# PATH CONFIG
BASE_DATASET = Path("all_dataset")
BASE_RAW     = BASE_DATASET / "raw_dataset"
BASE_CLEAN   = BASE_DATASET / "clean_dataset"

# Buat folder kalau belum ada
BASE_DATASET.mkdir(exist_ok=True)
BASE_RAW.mkdir(exist_ok=True)
BASE_CLEAN.mkdir(parents=True, exist_ok=True)

# LOAD FILES
def load_all_files(domain_name=None):
    files = list(BASE_RAW.glob("*.csv"))
    if domain_name:
        files = [f for f in files if domain_name in f.stem]
    if not files:
        print(f"[Warning] No CSV found for domain '{domain_name}'")
    data_dict = {f.stem: pd.read_csv(f) for f in files}
    return data_dict

In [17]:
# GENERAL CLEANING HELPERS
def to_datetime(df, cols):
    for c in cols:
        df[c] = pd.to_datetime(df[c], errors="coerce")
    return df

def to_numeric(df, cols):
    for c in cols:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

def clean_category(df, cols):
    for c in cols:
        df[c] = (
            df[c]
            .astype(str)
            .str.strip()
            .str.replace(r"\s+", "_", regex=True)
            .str.lower()
        )
    return df

def clip_numeric(df, col, min_val=None, max_val=None):
    if min_val is not None:
        df[col] = df[col].clip(lower=min_val)
    if max_val is not None:
        df[col] = df[col].clip(upper=max_val)
    return df

def validate_coordinates(df, lat_cols, lon_cols):
    for lat in lat_cols:
        df.loc[~df[lat].between(-11, 6), lat] = np.nan
    for lon in lon_cols:
        df.loc[~df[lon].between(95, 141), lon] = np.nan
    return df

In [18]:
# CLEANING FUNCTIONS

def clean_fleet(df):
    df = df.copy()
    df = to_datetime(df, ["week_start"])
    numeric_cols = [
    "available_trucks", "breakdown_trucks", "utilization_pct",
    "capacity_ton", "effective_capacity_ton", "tonnage_last_week"
]
    df = to_numeric(df, numeric_cols)
    df = clip_numeric(df, "utilization_pct", 0, 100)
    return df

def clean_heavy_equipment(df):
    df = df.copy()
    df = to_datetime(df, ["week_start"])
    numeric_cols = [
    "excavator_active", "dozer_active", "grader_active",
    "maintenance_units", "avg_operating_hours", "potential_breakdown_flag"
]
    df = to_numeric(df, numeric_cols)
    df = clip_numeric(df, "avg_operating_hours", 0, 24)
    return df

def clean_production(df):
    df = df.copy()
    df = to_datetime(df, ["week_start"])
    numeric_cols = numeric_cols = ["target_ton", "progress_ratio", "realized_ton"]
    df = to_numeric(df, numeric_cols)
    return df

def clean_road(df):
    df = df.copy()
    cat_cols = ["road_id", "segment_name", "segment_type", "surface_type", "curvature_level"]
    df = clean_category(df, cat_cols)
    numeric_cols = ["length_km", "slope_pct", "elevation_start_m", "elevation_end_m"]
    df = to_numeric(df, numeric_cols)
    df = clip_numeric(df, "slope_pct", 0, 35)
    df["elevation_gain_m"] = df["elevation_end_m"] - df["elevation_start_m"]
    df["is_climbing"] = (df["elevation_gain_m"] > 0).astype(int)
    df = to_numeric(df, ["lat_start", "lon_start", "lat_end", "lon_end"])
    df = validate_coordinates(df, ["lat_start", "lat_end"], ["lon_start", "lon_end"])
    return df

def clean_ship_schedule(df):
    df = df.copy()
    df = to_datetime(df, ["week_start", "eta", "etd"])
    df = clean_category(df, ["ship_size", "jetty_id"])
    numeric_cols = ["ship_size"] 
    df = to_numeric(df, numeric_cols)
    return df

def clean_stockpile(df):
    df = df.copy()
    df = clean_category(df, ["stockpile_id"])
    df = to_datetime(df, ["week_start"])
    numeric_cols = [
        "current_stock_ton", "incoming_production_ton",
        "planned_loading_ton", "stock_after_loading_ton"
    ]
    df = to_numeric(df, numeric_cols)
    return df

def clean_truck_to_ship(df):
    df = df.copy()
    df = to_datetime(df, ["week_start"])
    df = clean_category(df, ["truck_id", "jetty_id"])
    numeric_cols = [
        "allocated_for_shipping", "avg_cycle_time_min",
        "trip_count", "tonnage_moved_ton",
        "road_flood_flag", "crossing_queue_flag"
    ]
    df = to_numeric(df, numeric_cols)
    return df

def clean_weather(df):
    df = df.copy()
    df = to_datetime(df, ["date"])
    df = clean_category(df, ["location"])
    numeric_cols = [
        "lat", "lon", "rainfall_mm", "humidity_pct",
        "solar_radiation_wm2", "temp_c", "temp_max_c",
        "temp_min_c", "wind_speed_10m_mps", "cloud_cover_pct"
    ]
    df = to_numeric(df, numeric_cols)
    df = validate_coordinates(df, ["lat"], ["lon"])
    df = clip_numeric(df, "humidity_pct", 0, 100)
    df = clip_numeric(df, "cloud_cover_pct", 0, 100)
    return df

In [19]:
# LOAD RAW DATA
fleet_files           = load_all_files("fleet")
heavy_equipment_files = load_all_files("heavy_equipment")
production_files      = load_all_files("production")
weather_files         = load_all_files("weather")
road_files            = load_all_files("road")
truck_to_ship_files   = load_all_files("truck_to_ship")
ship_schedule_files   = load_all_files("ship_schedule")
stockpile_files       = load_all_files("stockpile")

In [20]:
# CLEAN & SAVE
domain_map = [
    ("fleet", fleet_files, clean_fleet),
    ("heavy_equipment", heavy_equipment_files, clean_heavy_equipment),
    ("production", production_files, clean_production),
    ("weather", weather_files, clean_weather),
    ("road", road_files, clean_road),
    ("truck_to_ship", truck_to_ship_files, clean_truck_to_ship),
    ("ship_schedule", ship_schedule_files, clean_ship_schedule),
    ("stockpile", stockpile_files, clean_stockpile)
]

for domain_name, files_dict, clean_func in domain_map:
    for fname, df in files_dict.items():
        try:
            df_clean = clean_func(df)
            # Save ke CSV
            df_clean.to_csv(BASE_CLEAN / f"{fname}.csv", index=False)
            print(f"[INFO] {fname} cleaned & saved successfully")
        except Exception as e:
            print(f"[ERROR] {fname} cleaning failed: {e}")

[INFO] fleet cleaned & saved successfully
[INFO] heavy_equipment cleaned & saved successfully
[INFO] production cleaned & saved successfully
[INFO] weather cleaned & saved successfully
[INFO] road cleaned & saved successfully
[INFO] truck_to_ship cleaned & saved successfully
[INFO] ship_schedule cleaned & saved successfully
[INFO] stockpile cleaned & saved successfully


In [21]:
# LOAD CLEAN DATA & VERIFY
datetime_cols = {
    "fleet": ["week_start"],
    "heavy_equipment": ["week_start"],
    "production": ["week_start"],
    "road": [],
    "truck_to_ship": ["week_start"],
    "ship_schedule": ["week_start", "eta", "etd"],
    "stockpile": ["week_start"],
    "weather": ["date"]
}

for f in BASE_CLEAN.glob("*.csv"):
    domain_name = f.stem.split("_")[0]  # misal 'fleet_xyz' → 'fleet'
    parse_cols = datetime_cols.get(domain_name, [])
    
    df = pd.read_csv(f, parse_dates=parse_cols)  # parse datetime
    print(f"\n=== File: {f.name} ===")
    print("Shape:", df.shape)
    print("Columns:", df.columns.tolist())
    print("Data types:\n", df.dtypes)
    print("\nHead:\n", df.head())
    print("\nMissing values:\n", df.isna().sum())


=== File: fleet.csv ===
Shape: (144, 9)
Columns: ['week_start', 'pit_id', 'available_trucks', 'breakdown_trucks', 'utilization_pct', 'capacity_ton', 'effective_capacity_ton', 'tonnage_last_week', 'predicted_repair_hours']
Data types:
 week_start                datetime64[ns]
pit_id                            object
available_trucks                   int64
breakdown_trucks                   int64
utilization_pct                  float64
capacity_ton                       int64
effective_capacity_ton           float64
tonnage_last_week                  int64
predicted_repair_hours           float64
dtype: object

Head:
   week_start pit_id  available_trucks  breakdown_trucks  utilization_pct  \
0 2023-01-02  PIT-1                24                 2            81.11   
1 2023-01-09  PIT-1                28                 3            91.19   
2 2023-01-16  PIT-1                18                 2            72.86   
3 2023-01-23  PIT-1                31                 0            81